In [62]:
import requests
from lxml import etree
import re
import json

In [66]:
falcon_url = "https://www.inoreader.com/stream/user/1004920918/tag/rw_monitor%3A%20FalconFeedsIO/view/json"

# fetch the RSS feed
response = requests.get(falcon_url)
rss_json = response.json()

In [69]:
rw_list = []
for item in rss_json['items']:
    tweet = item['title'].replace('\n',' ')
    ransomware = tweet.split('#ransomware')[0]
    date_published = item['date_published'][:10]
    date_published = pd.to_datetime(date_published).strftime('%Y-%m-%d')
    pattern = r"added (.*?) to their victim list"
    try:
        victim = re.search(pattern, tweet).group(1)
        if "," in victim:
            victim = victim.split(',')[0]
    except:
        victim = f'error: {link}'
            # print(f'Ransomware: {ransomware}\nVictim: {victim}\nLink: {link}\nDate: {date_published}')
    rw_list.append({'victim': victim,
    'ransomware': ransomware,
    'date_published':date_published,
    'link':link})
rw_df = pd.DataFrame(rw_list)
rw_df

,victim,ransomware,date_published,link
0,Palo Alto Sheriff (paloaltosheriff.com),PLAY,2023-04-09,https://twitter.com/falconfeedsio/status/16447...
1,PKF Antares (pkfantares.com),PLAY,2023-04-09,https://twitter.com/falconfeedsio/status/16447...
2,Legion Aero (legionaero.com),PLAY,2023-04-09,https://twitter.com/falconfeedsio/status/16447...
3,Michiels Vleeswaren,PLAY,2023-04-09,https://twitter.com/falconfeedsio/status/16447...
4,Schirm (schirm.com),PLAY,2023-04-09,https://twitter.com/falconfeedsio/status/16447...
5,Bayside Interiors (baysideinteriors.com),LockBit,2023-04-09,https://twitter.com/falconfeedsio/status/16447...
6,DCI Engineers (dci-engineers.com),Black Basta,2023-04-09,https://twitter.com/falconfeedsio/status/16447...


In [71]:
threatmon_url = 'https://www.inoreader.com/stream/user/1004920918/tag/rw_monitor%3A%20threatmon/view/json'

# fetch the RSS feed
response = requests.get(threatmon_url)
rss_json = response.json()

In [83]:
rw_list = []
for item in rss_json['items']:
    tweet = item['title'].replace('\n',' ')
    ransomware = tweet.split(' Victim :')[0].replace("Actor : ",'').strip()
    victim = tweet.split(' Victim : ')[1].split(" Date")[0].strip()
    date_published = pd.to_datetime(item['date_published'][:10]).strftime('%Y-%m-%d').strip()
    print(f'{ransomware} | {victim} | {date_published}')
#     date_published = item['date_published'][:10]
#     date_published = pd.to_datetime(date_published).strftime('%Y-%m-%d')
#     pattern = r"added (.*?) to their victim list"
#     try:
#         victim = re.search(pattern, tweet).group(1)
#         if "," in victim:
#             victim = victim.split(',')[0]
#     except:
#         victim = f'error: {link}'
#             # print(f'Ransomware: {ransomware}\nVictim: {victim}\nLink: {link}\nDate: {date_published}')
#     rw_list.append({'victim': victim,
#     'ransomware': ransomware,
#     'date_published':date_published,
#     'link':link})
# rw_df = pd.DataFrame(rw_list)
# rw_df

BlackByte | Creation Baumann | 2023-04-09
BlackByte | Crown Grinding & Machining | 2023-04-09
BlackByte | Cementos Bio-Bio | 2023-04-09
BlackByte | City of Collegedale | 2023-04-09
play | Schirm | 2023-04-09
play | Palo Alto County Sheriff | 2023-04-09
play | PKF Antares | 2023-04-09
play | Legion Aero | 2023-04-09
play | Vleeswarenfabriek Jac Michiels | 2023-04-09
LockBit | baysideinteriors.com | 2023-04-08
Black Basta (Basta News) | DCI-ENGINEERS_2 | 2023-04-08
BlackCat (ALPHV) | The Zalkin Law Firm P.C. | 2023-04-08
moneymessage | Pharmerica.com & BrightSpring Health Services | 2023-04-08
BlackCat (ALPHV) | SIVSA | 2023-04-07
LockBit | sxi.com.ph | 2023-04-07
BlackCat (ALPHV) | coremain | 2023-04-07
LockBit | 123.com | 2023-04-07
MoneyMessage | Micro Star International ( @msigaming ) | 2023-04-07
LockBit | baughmanco.com | 2023-04-07
Black Basta (Basta News) | HEICO | 2023-04-07
